In [ ]:
!git clone https://github.com/Vaishnav-Jayaram/CS6886w_Assignment1.git

Cloning into 'CS6886w_Assignment1'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [ ]:
%cd CS6886w_Assignment1/

/content/CS6886w_Assignment1


Setup & installs

In [1]:
# GPU check + installs (PyTorch CUDA is preinstalled on Colab)
import torch, platform, sys, importlib.metadata as im
print("Python:", sys.version)
print("Platform:", platform.platform())
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0), "| CUDA:", torch.version.cuda, "| cuDNN:", torch.backends.cudnn.version())

%pip -q install wandb==0.17.9 matplotlib==3.9.2 rich==13.9.2 -U scikit-learn
print("wandb:", im.version("wandb"))
print("sklearn:", im.version("scikit-learn"))
print("matplotlib:", im.version("matplotlib"))
print("rich:", im.version("rich"))

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
CUDA available: True
GPU: Tesla T4 | CUDA: 12.6 | cuDNN: 91002
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 133.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 144.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 135.2 MB/s eta 0:00:00
wandb: 0.17.9
sklearn: 1.7.2
matplotlib: 3.9.2
rich: 13.9.2


 Imports, seeding, device

In [2]:
import os, random, json, time, math, datetime
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import wandb

def set_seed(seed:int=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

def nowstamp():
    return datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

set_seed(42)

Using device: cuda


Data downloading & preprocessing (train aug, eval norm)

In [3]:
# CIFAR-10 normalization stats
CIFAR10_MEAN = (0.4914, 0.4822, 0.4465)
CIFAR10_STD  = (0.2470, 0.2435, 0.2616)

# Train aug + norm
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD),
])

# Val/Test norm
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN, CIFAR10_STD),
])

data_root = "./data"
train_full = datasets.CIFAR10(root=data_root, train=True,  download=True, transform=train_transform)
testset    = datasets.CIFAR10(root=data_root, train=False, download=True, transform=test_transform)

val_frac = 0.1
val_size = int(len(train_full)*val_frac)
train_size = len(train_full) - val_size
trainset, valset = random_split(train_full, [train_size, val_size], generator=torch.Generator().manual_seed(42))
print(f"Train/Val/Test sizes: {len(trainset)}/{len(valset)}/{len(testset)}")

100%|██████████| 170M/170M [00:44<00:00, 3.85MB/s]


Train/Val/Test sizes: 45000/5000/10000


Model: VGG6 (configurable activation)

In [4]:
ACTIVATIONS = {
    "relu": nn.ReLU(inplace=True),
    "sigmoid": nn.Sigmoid(),
    "tanh": nn.Tanh(),
    "silu": nn.SiLU(inplace=True),
    "gelu": nn.GELU(),
}

class VGG6(nn.Module):
    # 6 convs (64,64)→MP, (128,128)→MP, (256,256)→MP, then GAP+Linear
    def __init__(self, activation: str = "relu", num_classes: int = 10):
        super().__init__()
        act = ACTIVATIONS[activation]
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding=1), act,
            nn.Conv2d(64, 64, 3, padding=1), act,
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), act,
            nn.Conv2d(128, 128, 3, padding=1), act,
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1), act,
            nn.Conv2d(256, 256, 3, padding=1), act,
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(256, num_classes),
        )
    def forward(self, x):
        return self.classifier(self.features(x))

Optimizers

In [6]:
def build_optimizer(name:str, params, lr:float, weight_decay:float=5e-4, momentum:float=0.9):
    name = name.lower()
    if name == "sgd":
        return optim.SGD(params, lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=False)
    if name == "nesterov-sgd":
        return optim.SGD(params, lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=True)
    if name == "adam":
        return optim.Adam(params, lr=lr, weight_decay=weight_decay)
    if name == "adamw":
        return optim.AdamW(params, lr=lr, weight_decay=weight_decay)
    if name == "adagrad":
        return optim.Adagrad(params, lr=lr, weight_decay=weight_decay)
    if name == "rmsprop":
        return optim.RMSprop(params, lr=lr, momentum=momentum, weight_decay=weight_decay)
    if name == "nadam":
        return optim.NAdam(params, lr=lr, weight_decay=weight_decay)
    raise ValueError(f"Unknown optimizer: {name}")

Dataloaders & train/eval utilities

In [7]:
def make_loaders(batch_size:int=128, num_workers:int=2):
    tr = DataLoader(trainset, batch_size=batch_size, shuffle=True,  num_workers=num_workers, pin_memory=True)
    va = DataLoader(valset,   batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
    te = DataLoader(testset,  batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
    return tr, va, te

def accuracy_from_logits(logits, y):
    return (logits.argmax(dim=1) == y).float().mean().item()

def run_epoch(model, loader, criterion, optimizer=None):
    train_mode = optimizer is not None
    model.train(train_mode)
    losses, accs = [], []
    for xb, yb in loader:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        logits = model(xb)
        loss = criterion(logits, yb)
        if train_mode:
            optimizer.zero_grad(); loss.backward(); optimizer.step()
        losses.append(loss.item())
        accs.append(accuracy_from_logits(logits, yb))
    return float(np.mean(losses)), float(np.mean(accs))

@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    losses, accs = [], []
    for xb, yb in loader:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        logits = model(xb)
        loss = criterion(logits, yb)
        losses.append(loss.item())
        accs.append(accuracy_from_logits(logits, yb))
    return float(np.mean(losses)), float(np.mean(accs))

Baseline training (saves timestamped best weight in weights/, logs to W&B)

In [8]:
wandb.login()  # paste key or set WANDB_API_KEY before

default_config = {
    "activation": "relu",
    "optimizer": "nesterov-sgd",
    "batch_size": 128,
    "epochs": 30,
    "lr": 0.05,
    "momentum": 0.9,
    "weight_decay": 5e-4,
    "seed": 42,
}

from pathlib import Path
Path("weights").mkdir(exist_ok=True, parents=True)
Path("runs").mkdir(exist_ok=True, parents=True)

def train_one_run(config=None, project="vgg6-cifar10"):
    run_name = f"vgg6_{nowstamp()}"
    with wandb.init(project=project, config=config, name=run_name) as run:
        cfg = wandb.config
        set_seed(cfg.seed)

        # per-run meta
        run_dir = Path("runs") / f"{run.name or run.id}_{nowstamp()}"
        run_dir.mkdir(parents=True, exist_ok=True)
        (run_dir / "config.json").write_text(json.dumps(dict(cfg), indent=2))

        tr, va, te = make_loaders(cfg.batch_size)
        model = VGG6(activation=cfg.activation).to(DEVICE)
        criterion = nn.CrossEntropyLoss()
        opt = build_optimizer(cfg.optimizer, model.parameters(), lr=cfg.lr,
                              weight_decay=cfg.weight_decay, momentum=cfg.momentum)

        best_val, best_path = 0.0, None
        for epoch in range(cfg.epochs):
            tr_loss, tr_acc = run_epoch(model, tr, criterion, opt)
            va_loss, va_acc = evaluate(model, va, criterion)
            wandb.log({"epoch": epoch,
                       "train_loss": tr_loss, "train_acc": tr_acc*100.0,
                       "val_loss": va_loss,   "val_acc": va_acc*100.0})
            if va_acc > best_val:
                best_val = va_acc
                best_path = Path("weights") / f"{run.name or run.id}_{nowstamp()}_best.pt"
                torch.save(model.state_dict(), best_path)

        # test on best
        if best_path is not None and best_path.exists():
            model.load_state_dict(torch.load(best_path, map_location=DEVICE))
        te_loss, te_acc = evaluate(model, te, criterion)
        wandb.summary["test_acc"] = te_acc*100.0
        wandb.summary["test_loss"] = te_loss
        print(f"[{run.name}] Best Val={best_val*100:.2f}% | Test={te_acc*100:.2f}%")
        print("Best weight:", best_path)

        # upload weight artifact to W&B
        if best_path is not None and best_path.exists():
            art = wandb.Artifact(f"{run.name}_best", type="model",
                                 metadata={"val_acc": float(best_val), "test_acc": float(te_acc)})
            art.add_file(str(best_path))
            wandb.log_artifact(art)

# run baseline
train_one_run(default_config)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ee24d032 (ee24d032-iitm-india). Use `wandb login --relogin` to force relogin


[vgg6_20251025-183155] Best Val=86.78% | Test=85.83%
Best weight: weights/vgg6_20251025-183155_20251025-184111_best.pt


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_acc,▁▃▄▅▆▆▆▇▇▇▇▇▇▇████████████████
train_loss,█▆▅▄▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▃▄▅▅▆▆▇▇▇▇▇▇██▇███▇▇██▇▇█████
val_loss,█▆▅▄▄▃▃▂▂▂▂▂▂▁▁▂▂▁▁▂▂▁▁▂▂▁▁▁▁▁
epoch,29
test_acc,85.82872
test_loss,0.46008
train_acc,90.39319
train_loss,0.27841
val_acc,86.01562


W&B Sweep (creates Parallel Coordinates plot automatically). Set count=22.

In [9]:
sweep_config = {
    "method": "random",
    "metric": {"name": "val_acc", "goal": "maximize"},
    "parameters": {
        "activation":   {"values": ["relu", "silu", "gelu", "tanh", "sigmoid"]},
        "optimizer":    {"values": ["sgd", "nesterov-sgd", "adam", "adagrad", "rmsprop", "nadam"]},
        "batch_size":   {"values": [64, 128, 256]},
        "epochs":       {"values": [20, 30]},
        "lr":           {"values": [0.01, 0.02, 0.05, 0.1]},
        "momentum":     {"values": [0.0, 0.9]},
        "weight_decay": {"values": [0.0, 5e-4, 5e-3]},
        "seed":         {"values": [1, 2, 3]},
    }
}

wandb.login()
sweep_id = wandb.sweep(sweep_config, project="vgg6-cifar10")
def sweep_train(): train_one_run(project="vgg6-cifar10")
# EXACTLY 25 runs:
wandb.agent(sweep_id, function=sweep_train, count=22)

Create sweep with ID: cfaixdo8
Sweep URL: https://wandb.ai/ee24d032-iitm-india/vgg6-cifar10/sweeps/cfaixdo8


wandb: Agent Starting Run: 8cjqt4me with config:
wandb: 	activation: relu
wandb: 	batch_size: 256
wandb: 	epochs: 30
wandb: 	lr: 0.01
wandb: 	momentum: 0.9
wandb: 	optimizer: adam
wandb: 	seed: 2
wandb: 	weight_decay: 0.005
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[vgg6_20251025-184503] Best Val=19.39% | Test=19.65%
Best weight: weights/vgg6_20251025-184503_20251025-185416_best.pt


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▆█
train_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▆▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▁▁▂▁▂▁▁▂▁▁▁▁▁▁▁▂▁▁▁▁▁▂▁▂▂▂▄▆█
val_loss,███████████████████████████▇▅▁
epoch,29
test_acc,19.64844
test_loss,2.03794
train_acc,16.92605
train_loss,2.14867
val_acc,19.38764


wandb: Agent Starting Run: btj6ea39 with config:
wandb: 	activation: gelu
wandb: 	batch_size: 128
wandb: 	epochs: 30
wandb: 	lr: 0.05
wandb: 	momentum: 0.9
wandb: 	optimizer: rmsprop
wandb: 	seed: 1
wandb: 	weight_decay: 0.0005
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

[vgg6_20251025-211905] Best Val=10.37% | Test=10.21%
Best weight: weights/vgg6_20251025-211905_20251025-212214_best.pt


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_acc,█▅▁▃▃▃▆▆▄▄▄▅▃▃▄▄▄▃▃▄▃▃▅▃▅▄▁▃▄▃
train_loss,▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▇█▇▆█▇█▇██▇▇█▇▇▇▆▇▆▇█▆▇▇▆▆▆▆▆
val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,29
test_acc,10.20508
test_loss,2.3039
train_acc,9.8149
train_loss,2.30322
val_acc,9.38879


wandb: Agent Starting Run: t9r3i4rb with config:
wandb: 	activation: sigmoid
wandb: 	batch_size: 64
wandb: 	epochs: 30
wandb: 	lr: 0.05
wandb: 	momentum: 0.9
wandb: 	optimizer: sgd
wandb: 	seed: 1
wandb: 	weight_decay: 0.005
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


[vgg6_20251025-212835] Best Val=10.66% | Test=9.98%
Best weight: weights/vgg6_20251025-212835_20251025-213118_best.pt


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_acc,▅▆▄▅▆▃▅██▇▄▅▂▄▃▄▆▅▄▄█▆▆▁█▆▄▃▇▃
train_loss,█▁▂▂▂▃▂▁▁▁▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▂▂▂
val_acc,▅▅▁▅▂▅▂█▅█▅▅▁▄█▅▅▇▇▃▂▇▄▂▅▁▅▄▇█
val_loss,▂▅▄▆▇▃▄▁▃▃█▂▅▅▃▂▃▂▁▂▆▂▄▄▃▄▃▅▂▂
epoch,29
test_acc,9.98209
test_loss,2.30434
train_acc,9.84553
train_loss,2.3081
val_acc,10.6606


Re-run the best configuration (from W&B) & keep the weight

In [ ]:
best_config = {
    "activation": "silu",  # <-- replace from W&B best run
    "optimizer": "adamw",
    "batch_size": 128,
    "epochs": 30,
    "lr": 0.02,
    "momentum": 0.9,
    "weight_decay": 5e-4,
    "seed": 2,
}
# train_one_run(best_config)  # optional confirmatory run